<a href="https://colab.research.google.com/github/noritaketakamichi/manabie_auto_allocation_test/blob/main/auto_allocation_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================
# 1. ライブラリの準備とスプレッドシート接続
# ==========================================
!pip install ortools gspread pandas --quiet

import gspread
import pandas as pd
from google.colab import auth
from google.auth import default
from ortools.linear_solver import pywraplp

print("ライブラリの準備完了。認証を開始します...")

# Google認証
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# ▼▼▼ スプレッドシートのURL（必要に応じて変更してください） ▼▼▼
SPREADSHEET_URL = 'https://docs.google.com/spreadsheets/d/1NxNVevKlGBhTfUrVEhMcaUgy4WJWPBiObetuWYX_u70'
# ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲

try:
    # ワークブックを開く（wbという変数は次のセルでも使えます）
    wb = gc.open_by_url(SPREADSHEET_URL)
    print(f"✅ 接続成功: スプレッドシート '{wb.title}' を開きました。")
    print("次のセルを実行して、計算を開始してください。")
except Exception as e:
    print(f"❌ エラー: スプレッドシートが見つかりません。\n詳細: {e}")

ライブラリの準備完了。認証を開始します...
✅ 接続成功: スプレッドシート 'auto allocation test' を開きました。
次のセルを実行して、計算を開始してください。


In [ ]:
# ==========================================
# 2. データの読み込み・計算・書き出し (修正版)
# ==========================================

# ★シート名の定義
sheet_names = {
    'slots':         'lesson_slot',
    'courses':       'course',
    'students':      'student_list',
    'student_reqs':  'student_course',
    'student_avail': 'student_availability',
    'teachers':      'teacher_list',
    'teachable':     'teachable_courses',
    'teacher_avail': 'teacher_availability'
}

# データを格納する辞書
dfs = {}

try:
    print("--- データの読み込みを開始します ---")
    for key, sheet_name in sheet_names.items():
        try:
            ws = wb.worksheet(sheet_name)
            data = ws.get_all_records()
            dfs[key] = pd.DataFrame(data)
            print(f"OK: シート '{sheet_name}' ({len(dfs[key])}行)")
        except gspread.WorksheetNotFound:
            print(f"⚠️ 警告: シート '{sheet_name}' が見つかりません。")
            continue

    # データフレームセット
    df_slots = dfs.get('slots')
    df_courses = dfs.get('courses')
    df_students = dfs.get('students')
    df_student_reqs = dfs.get('student_reqs')
    df_student_avail = dfs.get('student_avail')
    df_teachers = dfs.get('teachers')
    df_teachable = dfs.get('teachable')
    df_teacher_avail = dfs.get('teacher_avail')

    if any(df is None or df.empty for df in [df_slots, df_students, df_teachers, df_student_reqs]):
        raise ValueError("必要なデータシートの一部が読み込めないか、空です。")

    # --- 前処理 ---
    student_map = dict(zip(df_students['id'], df_students['student_name']))
    teacher_map = dict(zip(df_teachers['id'], df_teachers['teacher_name']))
    course_map = dict(zip(df_courses['id'], df_courses['course_name']))

    slot_map = {}
    for _, row in df_slots.iterrows():
        slot_map[row['id']] = f"{row['date']} (Slot {row['time_range_id']})"

    teachable_dict = {}
    for _, row in df_teachable.iterrows():
        tid, cid = row['teacher_id'], row['course_id']
        if tid not in teachable_dict: teachable_dict[tid] = set()
        teachable_dict[tid].add(cid)

    student_avail_set = {}
    for _, row in df_student_avail.iterrows():
        sid, slid = row['student_id'], row['slot_id']
        if sid not in student_avail_set: student_avail_set[sid] = set()
        student_avail_set[sid].add(slid)

    teacher_avail_set = {}
    for _, row in df_teacher_avail.iterrows():
        tid, slid = row['teacher_id'], row['slot_id']
        if tid not in teacher_avail_set: teacher_avail_set[tid] = set()
        teacher_avail_set[tid].add(slid)

    requests = []
    for _, row in df_student_reqs.iterrows():
        requests.append((row['student_id'], row['course_id'], row['sessions']))

    all_slots = df_slots['id'].tolist()
    all_teachers = df_teachers['id'].tolist()

    print("データ準備完了。最適化計算を実行します...")

    # --- 最適化モデル作成 ---
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver: raise Exception("ソルバーの初期化に失敗しました")

    x = {}
    for sid, cid, sessions in requests:
        valid_teachers = [t for t in all_teachers if cid in teachable_dict.get(t, set())]
        for tid in valid_teachers:
            common_slots = student_avail_set.get(sid, set()).intersection(teacher_avail_set.get(tid, set()))
            for slid in common_slots:
                x[(sid, cid, tid, slid)] = solver.IntVar(0, 1, f'x_{sid}_{cid}_{tid}_{slid}')

    print(f"生成された変数の数: {len(x)}")

    # 制約1: 必要コマ数（★修正点：== ではなく <= にして、配置できない場合を許容する）
    for sid, cid, sessions in requests:
        relevant_vars = [v for (s, c, t, sl), v in x.items() if s == sid and c == cid]
        if relevant_vars:
            solver.Add(solver.Sum(relevant_vars) <= sessions)
        # 変数がない（空きがない）場合は自動的に0になるのでOK

    # 制約2: 生徒のダブルブッキング防止
    for sid in student_map.keys():
        for slid in all_slots:
            vars_s = [v for (s, c, t, sl), v in x.items() if s == sid and sl == slid]
            if vars_s: solver.Add(solver.Sum(vars_s) <= 1)

    # 制約3: 講師のダブルブッキング防止
    for tid in teacher_map.keys():
        for slid in all_slots:
            vars_t = [v for (s, c, t, sl), v in x.items() if t == tid and sl == slid]
            if vars_t: solver.Add(solver.Sum(vars_t) <= 1)

    # 目的関数（最大化）
    # たくさん授業を入れるほどスコアが高くなるため、可能な限り上限（sessions）まで埋めようとします
    objective = solver.Objective()
    for v in x.values(): objective.SetCoefficient(v, 1)
    objective.SetMaximization()

    # --- 計算実行 ---
    status = solver.Solve()

    if status in [pywraplp.Solver.OPTIMAL, pywraplp.Solver.FEASIBLE]:
        print("★ 計算完了！結果を集計します...")

        allocated_results = []
        allocated_counts = {} # {(sid, cid): count}

        # 1. 配置された授業の集計
        for (sid, cid, tid, slid), v in x.items():
            if v.solution_value() > 0.5:
                allocated_results.append([
                    slid, sid, tid, cid,
                    slot_map.get(slid, slid),
                    student_map.get(sid, sid),
                    teacher_map.get(tid, tid),
                    course_map.get(cid, cid)
                ])
                # カウントアップ
                key = (sid, cid)
                allocated_counts[key] = allocated_counts.get(key, 0) + 1

        # 2. 未配置（あまり）の集計
        unallocated_results = []
        for sid, cid, req_sessions in requests:
            actual_sessions = allocated_counts.get((sid, cid), 0)
            diff = req_sessions - actual_sessions

            if diff > 0:
                unallocated_results.append([
                    sid, cid, diff,
                    student_map.get(sid, sid),
                    course_map.get(cid, cid)
                ])

        # --- DataFrame作成 ---
        df_allocated = pd.DataFrame(allocated_results, columns=['slot_id', 'student_id', 'teacher_id', 'course_id', '日時', '生徒名', '講師名', '科目名'])
        df_allocated = df_allocated.sort_values(by=['slot_id', 'student_id'])

        df_unallocated = pd.DataFrame(unallocated_results, columns=['student_id', 'course_id', '不足コマ数', '生徒名', '科目名'])

        # 画面表示
        print("\n=== ✅ 配置成功リスト (一部) ===")
        display(df_allocated[['日時', '生徒名', '科目名', '講師名']].head())

        if not df_unallocated.empty:
            print(f"\n=== ⚠️ 未配置リスト ({len(df_unallocated)}件) ===")
            display(df_unallocated[['生徒名', '科目名', '不足コマ数']])
        else:
            print("\n=== ✨ 全ての授業が配置されました！ ===")

        # --- シート書き込み関数 ---
        def write_to_sheet(sheet_name, df):
            try:
                try:
                    ws = wb.worksheet(sheet_name)
                    ws.clear()
                except gspread.WorksheetNotFound:
                    ws = wb.add_worksheet(title=sheet_name, rows=1000, cols=10)

                if not df.empty:
                    ws.update([df.columns.values.tolist()] + df.values.tolist())
                    print(f"保存完了: '{sheet_name}'")
                else:
                    print(f"スキップ: '{sheet_name}' (データなし)")
            except Exception as e:
                print(f"エラー '{sheet_name}': {e}")

        # 書き出し実行
        write_to_sheet('output_allocated_lessons', df_allocated)
        write_to_sheet('output_unallocated_lessons', df_unallocated)

    else:
        print("❌ 計算に失敗しました。条件を見直してください。")

except Exception as e:
    print(f"\n【エラー発生】処理を中断しました。\n詳細: {e}")

--- データの読み込みを開始します ---
OK: シート 'lesson_slot' (12行)
OK: シート 'course' (4行)
OK: シート 'student_list' (6行)
OK: シート 'student_course' (9行)
OK: シート 'student_availability' (17行)
OK: シート 'teacher_list' (6行)
OK: シート 'teachable_courses' (24行)
OK: シート 'teacher_availability' (72行)
データ準備完了。最適化計算を実行します...
生成された変数の数: 150
★ 計算完了！結果を集計します...

=== ✅ 配置成功リスト (一部) ===


,日時,生徒名,科目名,講師名
0,2025-12-13 (Slot 1),s1,japanese,t1
4,2025-12-13 (Slot 1),s2,math,t2
1,2025-12-13 (Slot 2),s1,math,t1
5,2025-12-13 (Slot 2),s2,math,t2
14,2025-12-13 (Slot 2),s6,japanese,t3



=== ⚠️ 未配置リスト (3件) ===


,生徒名,科目名,不足コマ数
0,s2,science,2
1,s4,social,2
2,s6,japanese,1


保存完了: 'output_allocated_lessons'
保存完了: 'output_unallocated_lessons'
